In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import torch
import optuna
import os
import pytorch_lightning as pl

from Data.Drosophilla.FlyDataMod import FlyDataModule
from IPython.display import Image
from IPython.core.debugger import set_trace
from Models import LinearRegression as lrg
from Models import Transformer as tfm
from pytorch_lightning import LightningDataModule
from sklearn import preprocessing
from torch import nn as nn
from Utils import evaluations as ev
from Utils import callbacks as cb
from Utils import loggers as lg


In [ ]:
# Linear regression
label_type = 'gamma'
label_val  = 10

if not os.path.isdir("Temp_Regression"):
    os.mkdir("Temp_Regression")
    
trainer = pl.Trainer(
    gpus=1,
    max_epochs=5,
    callbacks=[cb.getcb()],
    default_root_dir="Temp_Regression")

dm = FlyDataModule(cell_line="S2",
                data_win_radius=200,
                batch_size=1,
                label_type=label_type,
                label_val=label_val)
dm.setup()
model = lrg.LinearRegressionModule(29,1)
#trainer.fit(model, dm)

In [ ]:
best_weight = 1
layer_weights = glob.glob("Temp_Regression/lightning_logs/version_"+str(best_weight)+"/checkpoints/*")[0]
print(layer_weights)
model = lrg.LinearRegressionModule.load_from_checkpoint(layer_weights).to("cuda:0")
dm    = FlyDataModule(cell_line="S2",
                  data_win_radius=5,
                  batch_size=1,
                  label_type=label_type,
                  label_val=label_val)
dm.setup()
model.cuda()
print(ev.getModelMetrics(model, dm, 'train'))
ev.createPlot(model, dm, 'train', "train.png")

In [ ]:
NUM_TOKEN     = 1
NUM_INP       = 29
NUM_HEAD      = 1
NUM_HID       = 2048
NUM_LAYERS    = 5
DROPOUT       = 0.2
OPTIM         = "Adam"
LEARNING_RATE = 0.001

torch.cuda.manual_seed_all(1)


In [ ]:
# Experiments with varying TAD Formation Metrics
label_types = ['insulation', 'insulation','insulation']
label_vals  = [3,5,10]

for label_type, label_val in zip(label_types, label_vals):
    def objective(trial):
        root_dir = "Experiments/ALT_TADS_TRANSFORMER/"+str(label_type)+"_"+str(label_val)
        if not os.path.isdir("Experiments/ALT_TADS_TRANSFORMER"):
            os.mkdir("Experiments/ALT_TADS_TRANSFORMER")
        if not os.path.isdir("Experiments/ALT_TADS_TRANSFORMER/"+str(label_type)+"_"+str(label_val)):
            os.mkdir(root_dir)
                 
        logger = lg.DictLogger(trial.number,
                       root_dir)
        trainer = pl.Trainer(
            logger=logger,
            gpus=1,
            max_epochs=50,
            callbacks=[cb.getcb()],
            default_root_dir=root_dir,
            terminate_on_nan=True)
        
        input_size = 29
        lr         = trial.suggest_categorical("lr", [1e-5, 1e-4, 1e-3, 1e-2, 1e-1])
        dropout    = trial.suggest_categorical("dropout", [0, 0.1, 0.2, 0.3])
        batch_size = trial.suggest_categorical("batch_size", [1,4,16,64])
        optimi     = trial.suggest_categorical("optimi", ["Adam", "SGD"])
        nlayers    = trial.suggest_categorical("nlayers", [1,2,3,4,5])
    
        dm = FlyDataModule(cell_line="S2",
                      data_win_radius=5,
                      batch_size=batch_size,
                      label_type=label_type,
                      label_val=label_val)
        dm.setup()
        model = tfm.TransformerModule(
            ntoken=1,
            ninp=29,
            nhead=1,
            nhid=2048,
            nlayers=nlayers,
            dropout=dropout,
            optimi=optimi,
            lr=lr,
            loss_type="mse")
        trainer.fit(model, dm)
        score = logger.metrics[-1]['val weighted mse loss']
        if np.isnan(score):
            score= 9999999999
        return score

    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=2)
    print("number of finished trials: {}".format(len(study.trials)))
    print("Best Trial")
    trial = study.best_trial
    print(trial)

In [ ]:
label_type  = "insulation"
label_val   = 3
best_weight = 0
layer_weights = glob.glob("Experiments/ALT_TADS_TRANSFORMER/"+str(label_type)+"_"+str(label_val)+"/optuna/version_"+str(best_weight)+"/checkpoints/*")[0]
#model = tfm.TransformerModule.load_from_checkpoint(layer_weights).to("cuda:0")
dm    = FlyDataModule(cell_line="S2",
                  data_win_radius=5,
                  batch_size=1,
                  label_type=label_type,
                  label_val=label_val)
dm.setup()
model.cuda()
print(ev.getModelMetrics(model, dm, 'test'))
ev.createPlot(model, dm, 'test', "test.png")